In [1]:
import os
import pandas as pd
import numpy as np
import gensim
from gensim import corpora, models
from nltk.corpus import stopwords

from string import punctuation

**gensim** - библиотека обработки естественного языка предназначения для «Тематического моделирования»
https://pythonru.com/biblioteki/gensim
https://radimrehurek.com/gensim/auto_examples/index.html
- *corpora* - предоставляет методы для преобразование текста в векторное представление

**nltk** - библиотека Python для обработки естественного языка
https://www.nltk.org/

**string.punctuation** возвращает все наборы знаков препинания
https://www.geeksforgeeks.org/string-punctuation-in-python/





Чтение датасета

In [4]:
dataset_tweets = pd.read_csv('data_elonmusk.csv', encoding='latin1')

dataset_tweets

,row ID,Tweet,Time,Retweet from,User
0,Row0,@MeltingIce Assuming max acceleration of 2 to ...,2017-09-29 17:39:19,NaN,elonmusk
1,Row1,RT @SpaceX: BFR is capable of transporting sat...,2017-09-29 10:44:54,SpaceX,elonmusk
2,Row2,@bigajm Yup :),2017-09-29 10:39:57,NaN,elonmusk
3,Row3,Part 2 https://t.co/8Fvu57muhM,2017-09-29 09:56:12,NaN,elonmusk
4,Row4,Fly to most places on Earth in under 30 mins a...,2017-09-29 09:19:21,NaN,elonmusk
...,...,...,...,...,...
3213,Row3213,"@YOUSRC Amos's article was fair, but his edito...",2012-11-20 08:52:03,NaN,elonmusk
3214,Row3214,These articles in Space News describe why Aria...,2012-11-20 08:38:31,NaN,elonmusk
3215,Row3215,Was misquoted by BBC as saying Europe's rocket...,2012-11-20 08:30:44,NaN,elonmusk
3216,Row3216,Just returned from a trip to London and Oxford...,2012-11-19 08:59:46,NaN,elonmusk


Препроцессинг

In [ ]:
#Формируем корпус из твитов. Готовим данные в корректном формате для передачи их в качестве входных параметров модели LDA

corpus=[]
for i in range(len(dataset_tweets['Tweet'])):
        corpus.append(dataset_tweets['Tweet'][i])
corpus

In [6]:
#Загружаем стоп-слова
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1

#удаляем стоп-слова, знаке препинания и другие не влияющие на смысл слова
texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

#Преобразуем данные в словарь, представляющий собой коллекцию уникальных токенов
dictionary = corpora.Dictionary(texts)

texts

**dictionary.doc2bow** подсчитывает число вхождений и генерирует целочисленный идентификатор для каждого слова. Результат возвращается в виде разреженного вектора.


In [17]:
#Создаем корпус Bag of Words
corpus = [dictionary.doc2bow(text) for text in texts]

#Применяем tfidf модель для удаления часто встречающих, но не влияющих на основую суть текста, слова
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf

In [18]:
#Обучаем LDA модель
total_topics = 5

lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf]

In [20]:
#Вывод результатов

lda.show_topics(total_topics,20)

[(0,
  '0.004*"tesla" + 0.003*"much" + 0.003*"one" + 0.003*"space" + 0.003*"find" + 0.002*"like" + 0.002*"would" + 0.002*"ny" + 0.002*"also" + 0.002*"earth" + 0.002*"definitely" + 0.002*"good" + 0.002*"station" + 0.002*"rear" + 0.002*"model" + 0.002*"cool" + 0.002*"work" + 0.002*"people" + 0.002*"coming" + 0.002*"car"'),
 (1,
  '0.008*"rocket" + 0.007*"launch" + 0.006*"good" + 0.004*"falcon" + 0.004*"9" + 0.003*"thanks" + 0.003*"flight" + 0.003*"@spacex:" + 0.003*"landing" + 0.003*":)" + 0.003*"dragon" + 0.003*"space" + 0.003*"@spacex" + 0.002*"like" + 0.002*"make" + 0.002*"air" + 0.002*"spacex" + 0.002*"tesla" + 0.002*"almost" + 0.002*"orbit."'),
 (2,
  '0.008*"tesla" + 0.006*"falcon" + 0.006*"new" + 0.004*"9" + 0.004*"great" + 0.004*"launch" + 0.004*"first" + 0.004*"@spacex:" + 0.003*":)" + 0.003*"rocket" + 0.003*"next" + 0.003*"long" + 0.003*"@teslamotors" + 0.003*"time" + 0.003*"i\'m" + 0.003*"#dragon" + 0.002*"coming" + 0.002*"@spacex" + 0.002*"energy" + 0.002*"dragon"'),
 (3,
  '

In [21]:
data_lda = {i: dict(lda.show_topic(i,25)) for i in range(total_topics)}
data_lda

{0: {'tesla': 0.0042598224,
  'much': 0.003341169,
  'one': 0.0030169426,
  'space': 0.00267355,
  'find': 0.0025259103,
  'like': 0.0024547256,
  'would': 0.002137063,
  'ny': 0.0020987038,
  'also': 0.002088472,
  'earth': 0.002056803,
  'definitely': 0.0020399364,
  'good': 0.001992751,
  'station': 0.0019830107,
  'rear': 0.0019546645,
  'model': 0.0018646172,
  'cool': 0.0018309772,
  'work': 0.0018161389,
  'people': 0.0018152193,
  'coming': 0.0018091308,
  'car': 0.0016998297,
  'auto': 0.0016837557,
  'dragon': 0.0016465181,
  'satellite': 0.001544126,
  'falcon': 0.0015021424,
  'pretty': 0.0014818053},
 1: {'rocket': 0.007827858,
  'launch': 0.0066516064,
  'good': 0.0055094245,
  'falcon': 0.004381834,
  '9': 0.0038422334,
  'thanks': 0.0034551115,
  'flight': 0.0033586791,
  '@spacex:': 0.0032148494,
  'landing': 0.003037769,
  ':)': 0.0028968768,
  'dragon': 0.0028606334,
  'space': 0.0025814734,
  '@spacex': 0.0025108503,
  'like': 0.002399503,
  'make': 0.0021933797,
  

In [22]:
df_lda = pd.DataFrame(data_lda)
df_lda = df_lda.fillna(0).T

df_lda

,tesla,much,one,space,find,like,would,ny,also,earth,...,best,high,change,live,read,cars,climate,made,public,x
0,0.004260,0.003341,0.003017,0.002674,0.002526,0.002455,0.002137,0.002099,0.002088,0.002057,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
1,0.001897,0.000000,0.001651,0.002581,0.000000,0.002400,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
2,0.008465,0.000000,0.000000,0.002351,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
3,0.008973,0.000000,0.000000,0.000000,0.000000,0.005688,0.003735,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
4,0.015479,0.000000,0.002387,0.000000,0.000000,0.003311,0.000000,0.000000,0.000000,0.000000,...,0.004371,0.002436,0.002334,0.002318,0.002287,0.00226,0.00226,0.002247,0.002081,0.001973


In [24]:
! pip install pyLDAvis
! pip install "pandas<2.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 34.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicor

**pyLDAvis** визуализирует результаты модели LDA

https://pyldavis.readthedocs.io/en/latest/modules/API.html

In [27]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

panel = gensimvis.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
3     -39.681271   2.958856       1        1  33.797430
0      35.463249 -41.138134       2        1  30.267529
4     -28.560455 -60.616047       3        1  15.751783
1     -10.304770  60.499069       4        1  10.093380
2      44.765392  22.620508       5        1  10.089878, topic_info=         Term       Freq      Total Category  logprob  loglift
580     model  20.000000  20.000000  Default  30.0000  30.0000
231     tesla  25.000000  25.000000  Default  29.0000  29.0000
90     rocket  11.000000  11.000000  Default  28.0000  28.0000
329    launch  11.000000  11.000000  Default  27.0000  27.0000
82     falcon   8.000000   8.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
2914      via   0.573889   2.107647   Topic5  -6.3382   0.9927
240      need   0.640277   4.041321   Topic5  -6.2287   0.4512
328   landing   0.698761   7.712071   Topic5  -6.1413  -0.1076
383        us   0.593041   3.286305   Topic5  -6.3054   0.5814
725     right   0.592090   3.362926   Topic5  -6.3070   0.5567

[365 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1013      5  0.853223  #dragon
6249      3  1.372955   #tesla
4566      2  1.468089      (ie
5547      3  1.614560      (no
7934      1  1.071063     (not
...     ...       ...      ...
814       1  0.355990        x
814       3  0.355990        x
153       1  0.797598      yes
153       2  0.265866      yes
2695      2  0.780964     yet.

[329 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 5, 2, 3])

Методы улучшения:
- Сделать более качественную предобработку текста (удалить ненужные символы, упоминания)
- Создать биграммы, триграммы
- Выделить ключевые слова (с помощью методов автоматического веделения терминов ATE)
- Настроить гиперпараметры

